In [2]:
from datasets import load_from_disk
import PyPDF2
from docx import Document
from datasets import Dataset
from datasets.dataset_dict import DatasetDict
import os
from docx.shared import Pt
# dataset = load_from_disk("data/cleaned-data-mc4-v2")
# dataset = load_from_disk("larger-data-model/modified-mc4")
# dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2260068
    })
    test: Dataset({
        features: ['text'],
        num_rows: 565018
    })
})

In [2]:
# type(dataset)

In [2]:
from datasets import concatenate_datasets, load_from_disk
d = {'train':Dataset.from_dict({'label':['something'],'text':['something']}),
     'test':Dataset.from_dict({'label':['something'],'text':['something']})
     }
t = {'train': Dataset.from_dict({'label':['something1'],'text':['something1']})}
d = DatasetDict(d)
# d['train']
eval_dataset = concatenate_datasets([d["test"], t['train']])
# d['train'][0]['text']
eval_dataset

Dataset({
    features: ['label', 'text'],
    num_rows: 2
})

In [3]:
def read_pdf(file):
    pdf = open(file, 'rb')
    reader = PyPDF2.PdfReader(pdf)
    dataset = []

    for page in range(1, len(reader.pages)):
        dataset.append({'text': reader.pages[page].extract_text()})

    return dataset

def read_docx(file):
    document = Document(file)
    text = []
    for line in document.paragraphs:
        text.append(line.text)

    return [{'text': t} for t in text]




# Function to convert the text to a single font
def convert_to_single_font(text, font_name, font_size):
    georgian_mapping = {
        'a': 'ა', 'b': 'ბ', 'g': 'გ', 'd': 'დ', 'e': 'ე', 'v': 'ვ', 'z': 'ზ', 't': 'თ', 'i': 'ი', 'k': 'კ',
        'l': 'ლ', 'm': 'მ', 'n': 'ნ', 'o': 'ო', 'p': 'პ', 'zh': 'ჟ', 'r': 'რ', 's': 'ს', 't': 'ტ', 'u': 'უ',
        'f': 'ფ', 'q': 'ქ', 'gh': 'ღ', 'y': 'ყ', 'sh': 'შ', 'ch': 'ჩ', 'ts': 'ც', 'dz': 'ძ', 'tz': 'წ', 'tch': 'ჭ',
        'kh': 'ხ', 'j': 'ჯ', 'h': 'ჰ', ' ': ' '
    }

    doc = Document()
    paragraph = doc.add_paragraph()

    run = paragraph.add_run()
    font = run.font
    font.name = font_name
    font.size = Pt(font_size)

    converted_text = ''
    for char in text:
        for c in char:
            lowercase_char = char.lower()
            georgian_char = georgian_mapping.get(lowercase_char, lowercase_char)
            converted_text += georgian_char

    return converted_text


In [4]:
# assign directory
directory = ['data/book-collection-elibrary/', 'data/books/']

def read_books(directory_list):
    book_datasets = []
    for directory in directory_list:
        for entry in os.scandir(directory):
            if entry.is_file():
                if entry.name.endswith('.pdf'):
                    book_datasets.extend(read_pdf(entry.path))
                elif entry.name.endswith('.docx'):
                    book_datasets.extend(read_docx(entry.path))
            elif entry.is_dir():
                book_datasets.extend(read_books([entry.path]))

    return Dataset.from_dict({'text': [d['text'] for d in book_datasets]})




In [5]:
new_dict = read_books(directory)
new_dict


KeyboardInterrupt



In [22]:
new_dict

Dataset({
    features: ['text'],
    num_rows: 527256
})

In [6]:
from datasets import concatenate_datasets
new_dataset = concatenate_datasets([dataset['train'], new_dict, dataset['test'], dataset['validation']])
new_dataset

Dataset({
    features: ['text'],
    num_rows: 2825086
})

In [7]:
bigger_dataset = new_dataset.train_test_split(test_size=0.2)
bigger_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2260068
    })
    test: Dataset({
        features: ['text'],
        num_rows: 565018
    })
})

In [3]:
from datasets import load_from_disk
from transformers import TFAutoModelForMaskedLM, DistilBertTokenizerFast
import re

# greek = r"[\u0370-\u03FF]+"
# chinese = r"[\u4E00-\u9FFF。]+"
# italic = r"[\U0001D400-\U0001D7FF]+"
# hindi = r"[\u0900-\u097F]+"
# telugu = r"[\u0C00-\u0C7F]+"
# kannada = r"[\u0C80-\u0CFF]+"
# odia = r"[\u0B00-\u0B7F]+"
# bengali = r"[\u0980-\u09FF]+"
# geez = r"[\u1200-\u137F]+"
# malayalam = r"[\u0D00-\u0D7F]+"
# russian = "[а-яА-ЯёЁ]+"
# emojis = r"[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF]+"
# hangul = r"[\uAC00-\uD7AF\u1100-\u11FF\u3130-\u318F\uA960-\uA97F\uAC00-\uD7A3]+"
# arabic = r"[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFDFF\uFE70-\uFEFF]+"
# additional = r"[\u2190-\u2BFF\u3000-\u303F\u25A0-\u27BF\ufe00-\ufe0f\ufe30-\ufe6f\U0001F300-\U0001F5FF]+"
# punctuation = r'[^\w\s]'
# latin_1 = r"[ª²³º¼½¾ßæðøþαεηιορυω]"
# latin_b = r"[đħıĳĸŀłŉŋœŧſƀƃƅƈƌƍƒƕƙƚƛƞƣƥƨƪƫƭƴƶƹƺƻƾƿǀǁǂǃǆǉǝǥǳȝȡȣȥȴȵȶȷȼȿɀɂɇɉɋɍɏɐɑɒɓɔɕɖɗɘəɚɛɜɝɞɟɠɡɢɣɤɥɦɧɨɩɪɫɬɭɮɯɰɱɲɳɴɵɶɷɸɹɺɻɼɽɾɿʀʁʂʃʄʅʆʇʈʉʊʋʌʍʎʏʐʑʒʓʔʕʖʗʘʙʚʛʜʝʞʟʠʡʢʣʤʥʦʧʨʩʪʫʬʭʮʯʰʱʲʳʴʵʶʷʸʹʺʻʼʽʾʿˀˁˆˇˈˉˊˋˌˍˎˏːˑˠˡˢˣˤˬˮ]"
# cryillic = r"[а-яА-ЯёЁђєѕіјљњћџѡѣѥѧѩѫѭѯѱѳѵѹѻѽѿҁҋҍҏґғҕҗҙқҝҟҡңҥҧҩҫҭүұҳҵҷҹһҽҿӄӆӈӊӌӎӏӕәӡөӷӻӽӿԁԃԅԇԉԋԍԏԑԓԕԗԙԛԝԟԡԣԥԧԩԫԭԯՙաբգդեզէըթժիլխծկհձղճմյնշոչպջռսվտրցւփքօֆև]"


# combined_pat = r'|'.join((greek, italic, chinese, hindi, telugu, kannada, odia, bengali, geez, malayalam, emojis, russian, hangul, arabic, additional, punctuation, r"\n"))

def remove_non_georgian_english_numbers(text):
    # Define a regular expression pattern to match non-Georgian, non-English, and non-number characters
    pattern = r"[^a-zA-Zა-ჰ0-9\s]+"

    # Use the sub() function from the re module to replace non-matching characters with an empty string
    cleaned_text = re.sub(pattern, " ", text)

    # Remove leading and trailing spaces
    cleaned_text = cleaned_text.strip()

    # Reduce consecutive spaces to a single space
    cleaned_text = re.sub(r"\s+", " ", cleaned_text)

    return cleaned_text

def clean_dataset(example):
    return {'text': remove_non_georgian_english_numbers(example['text'])}

In [8]:
cleaned_dataset = dataset.map(clean_dataset)
cleaned_dataset

Map:   0%|          | 0/2260068 [00:00<?, ? examples/s]

Map:   0%|          | 0/565018 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2260068
    })
    test: Dataset({
        features: ['text'],
        num_rows: 565018
    })
})

In [10]:
cleaned_dataset.save_to_disk('larger-data-model/modified-mc4-v2')

Saving the dataset (0/28 shards):   0%|          | 0/2260068 [00:00<?, ? examples/s]

Saving the dataset (0/7 shards):   0%|          | 0/565018 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer

dataset = load_from_disk('larger-data-model/modified-mc4-v2')
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def get_training_corpus():
    train_dataset = dataset["train"]
    for start_idx in range(0, len(train_dataset), 1000):
        samples = train_dataset[start_idx : start_idx + 1000]
        yield samples["text"]


training_corpus = get_training_corpus()

new_tokenizer = tokenizer.train_new_from_iterator(training_corpus, 30500)

In [4]:
new_tokenizer.save_pretrained("larger-data-model/distilbert-tokenizer")
tokenizer = new_tokenizer
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30500, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [5]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


tokenized_datasets = dataset.map(
    tokenize_function, batched=True, remove_columns=["text"]
)
tokenized_datasets.save_to_disk('larger-data-model/tokenized-dataset')
tokenized_datasets

Map:   0%|          | 0/2260068 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (611 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/565018 [00:00<?, ? examples/s]

Saving the dataset (0/28 shards):   0%|          | 0/2260068 [00:00<?, ? examples/s]

Saving the dataset (0/7 shards):   0%|          | 0/565018 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 2260068
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 565018
    })
})

In [6]:
chunk_size = 512
def group_texts(examples):
    # შევაწებოთ ტექსტები
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # გამოვთვალოთ გაერთიანებული მონაცემების სიგრძე
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # თუ სხვებზე პატარაა ბოლო ნაწილი მოდელში არ შევუშვათ
    total_length = (total_length // chunk_size) * chunk_size
    # მითითებული სიგრძის მიხედვით დავანაწევროდ
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # შევქმნათ ახალი სვეტი labels
    result["labels"] = result["input_ids"].copy()
    return result

In [7]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets.save_to_disk('larger-data-model/lm_dataset')
lm_datasets

Map:   0%|          | 0/2260068 [00:00<?, ? examples/s]

Map:   0%|          | 0/565018 [00:00<?, ? examples/s]

Saving the dataset (0/45 shards):   0%|          | 0/2059516 [00:00<?, ? examples/s]

Saving the dataset (0/12 shards):   0%|          | 0/511608 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2059516
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 511608
    })
})

In [4]:
from transformers import DataCollatorForLanguageModeling, DataCollatorForWholeWordMask
tokenizer = DistilBertTokenizerFast.from_pretrained('larger-data-model/distilbert-tokenizer')
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
whole_word_collator = DataCollatorForWholeWordMask(tokenizer=tokenizer, mlm_probability=0.15)

In [5]:
from transformers import TFAutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)
lm_datasets = load_from_disk('larger-data-model/lm_dataset')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


In [6]:
tf_train_dataset = model.prepare_tf_dataset(
    lm_datasets["train"],
    collate_fn=whole_word_collator,
    shuffle=True,
    batch_size=32,
)

tf_eval_dataset = model.prepare_tf_dataset(
    lm_datasets["test"],
    collate_fn=whole_word_collator,
    shuffle=False,
    batch_size=32,
)

C:\Users\Davit6174\PycharmProjects\pythonProject\venv\lib\site-packages\transformers\data\data_collator.py:951: UserWarning: DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. Please refer to the documentation for more information.
  warnings.warn(


In [7]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

model_name = model_checkpoint.split("/")[-1]

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4090, compute capability 8.9


In [8]:
model.fit(tf_train_dataset, validation_data=tf_eval_dataset)

28859/64359 [============>.................] - ETA: 3:40:10 - loss: 6.3037

In [9]:
model.save_pretrained('larger-data-model/distilbert-base-uncased-fine-tuned')

In [10]:
import math

eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

In [11]:
f = open("perplexity-large.txt", "w")
f.write(f"Perplexity of model_large: {math.exp(eval_loss):.2f}")
f.close()